# importing the MLBox

In [1]:
from mlbox.preprocessing import *
from mlbox.optimisation import *
from mlbox.prediction import *

/home/datageek/anaconda2/lib/python2.7/site-packages/mlbox/preprocessing/drift/__init__.py:9: UserWarning: ipCluster is starting. Please wait 30 sec and check in terminal that 'the engines appear to have started successfully'.
  warnings.warn("ipCluster is starting. Please wait 30 sec and check in terminal that 'the engines appear to have started successfully'.")
Using Theano backend.
/home/datageek/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
sep = ','
paths = ["train.csv","test.csv"]
target_name="Survived"

scoring = 'accuracy'
n_folds=3

# reading and cleaning all files 

In [3]:
rd = Reader(sep)
df = rd.train_test_split(paths, target_name)


reading csv : train.csv ...
cleaning data...
CPU time: 0.077516078949 seconds

reading csv : test.csv ...
cleaning data...
CPU time: 0.0522921085358 seconds

number of common features : 11

gathering and crunching for train and test datasets
reindexing for train and test datasets
dropping training duplicates
dropping constant variables on training set

number of categorical features : 5
number of numerical features : 6
number of training samples : 891
number of test samples : 418

task : classification
0.0    549
1.0    342
Name: Survived, dtype: int64
encoding target


In [4]:
dft = Drift_thresholder()
df = dft.fit_transform(df)


computing drifts...
CPU time: 0.318742990494 seconds

Top 10 drifts

('PassengerId', 1.0)
('Name', 0.99704442935827498)
('Ticket', 0.84579063327276627)
('Cabin', 0.59550700418006364)
('Embarked', 0.53834737054254522)
('SibSp', 0.52134352316754273)
('Fare', 0.51744001298932041)
('Sex', 0.51694417436933204)
('Pclass', 0.51382143975575834)
('Parch', 0.50992009325888499)

deleted variables : ['Name', 'PassengerId', 'Ticket']

dumping drift coefficients into directory : save
drift coefficients dumped


# tuning

In [5]:
opt = Optimiser(scoring, n_folds)

In [6]:
opt.evaluate(None, df)

No parameters set. Default configuration is tested

########################################################## testing hyper-parameters... #################################################################

>>> NA ENCODER :{'numerical_strategy': 'mean', 'categorical_strategy': '<NULL>'}

>>> CA ENCODER :{'strategy': 'label_encoding'}

>>> ESTIMATOR :{'sigmoid': 1.0, 'seed': 0, 'max_drop': 50, 'min_split_gain': 0, 'boosting_type': 'gbdt', 'silent': True, 'min_child_samples': 10, 'nthread': -1, 'min_child_weight': 5, 'strategy': 'LightGBM', 'use_missing': True, 'uniform_drop': False, 'max_depth': -1, 'skip_drop': 0.5, 'num_leaves': 31, 'reg_alpha': 0, 'subsample_for_bin': 50000, 'scale_pos_weight': 1, 'learning_rate': 0.05, 'max_bin': 255, 'n_estimators': 500, 'objective': 'binary', 'colsample_bytree': 0.8, 'subsample_freq': 1, 'xgboost_dart_mode': False, 'subsample': 0.9, 'reg_lambda': 0, 'drop_rate': 0.1, 'is_unbalance': False}


MEAN SCORE : accuracy = 0.806958473625
VARIANCE : 0.01514

0.80695847362514028

In [9]:
space = {
    
        'ne__numerical_strategy':{"search":"choice",
                                 "space":[0]},
        'ce__strategy':{"search":"choice",
                        "space":["label_encoding","random_projection", "entity_embedding"]}, 
        'fs__threshold':{"search":"uniform",
                        "space":[0.01,0.3]},    
        'est__max_depth':{"search":"choice",
                                  "space":[3,4,5,6,7]}
    
        }

best = opt.optimise(space, df,15)


########################################################## testing hyper-parameters... #################################################################

>>> NA ENCODER :{'numerical_strategy': 0, 'categorical_strategy': '<NULL>'}

>>> CA ENCODER :{'strategy': 'random_projection'}

>>> FEATURE SELECTOR :{'threshold': 0.17105993661756697, 'strategy': 'l1'}

>>> ESTIMATOR :{'sigmoid': 1.0, 'seed': 0, 'max_drop': 50, 'min_split_gain': 0, 'boosting_type': 'gbdt', 'silent': True, 'min_child_samples': 10, 'nthread': -1, 'min_child_weight': 5, 'strategy': 'LightGBM', 'use_missing': True, 'uniform_drop': False, 'max_depth': 5, 'skip_drop': 0.5, 'num_leaves': 31, 'reg_alpha': 0, 'subsample_for_bin': 50000, 'scale_pos_weight': 1, 'learning_rate': 0.05, 'max_bin': 255, 'n_estimators': 500, 'objective': 'binary', 'colsample_bytree': 0.8, 'subsample_freq': 1, 'xgboost_dart_mode': False, 'subsample': 0.9, 'reg_lambda': 0, 'drop_rate': 0.1, 'is_unbalance': False}


MEAN SCORE : accuracy = 0.793490460

In [10]:
prd = Predictor()
prd.fit_predict(best,df)


fitting the pipeline...
CPU time: 0.176958799362 seconds

predicting...
CPU time: 0.0571658611298 seconds

top 10 predictions :

        0.0       1.0  Survived_predicted
0  0.951520  0.048480                   0
1  0.915900  0.084100                   0
2  0.929849  0.070151                   0
3  0.744718  0.255282                   0
4  0.736510  0.263490                   0
5  0.680616  0.319384                   0
6  0.636542  0.363458                   0
7  0.953092  0.046908                   0
8  0.168989  0.831011                   1
9  0.981171  0.018829                   0

dumping predictions into directory : save


<mlbox.prediction.predictor.Predictor instance at 0x7fb4546365a8>